# 05. Fonctionalités avancées

Dans ce cours:
- Déguiser son scraper

Puis, sur la base d'exemples:
- Les différents types de requêtes
- Les API privées
- Les sessions

## Déguiser son scraper

Les requêtes HTTP contiennent de nombreuses informations, notamment concernant le navigateur et la plateforme utilisée. C'est le champ *user-agent* des en-têtes (headers) de la requête.

Un user-agent:

`Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0`

Indique que la page est demandée par un navigateur Firefox (version 60) sur un ordinateur équipé du système d'exploitation Ubuntu (Linux).

Certains serveurs utilisent ce champ pour vérfier qui demande la page (par exemple pour servir une version différente pour Google, pour un internaute sur iPhone ou pour un internaute sur PC).

Requests, notre librairie qui effectue les requêtes HTTP, n'envoit par défaut aucun user-agent, ce qui peut paraître suspect.

Voilà [une liste de user-agents](https://deviceatlas.com/blog/list-of-user-agent-strings).

In [ ]:
# Pour modifier son user-agent, on utilise le paramètre headers lors de la requête de la page

headers = {
    # Ce user-agent indique Chrome 47 sur un ordinateur Windows 7
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"
}

r = requests.get("http://google.com", headers = headers)

print(r.status_code)

## Exemple 1: Ellis Island et les requêtes GET et POST

Le site que nous scrapons à présent contient une base de données de tous les passagers accueillis à Ellis Islands (immigrants et touristes) entre 1892 et 1954.

https://www.libertyellisfoundation.org/passenger-result

On peut faire une recherche sur le lieu d'origine, mais on est toujours obligé d'indiquer au moins deux lettres du nom de famille.

Et l'URL des pages de résultat reste le même.

Ouvrons la console...

### Requêtes GET et POST

Les requêtes HTTP peuvent être de différentes sortes. En scraping, on utilise surtout les requêtes GET et POST.

Les requêtes GET sont celles que nous avons utilisées depuis le début. Ce sont celles par défaut, celles que l'on fait lorsque l'on tape une adresse dans la barre d'URL du navigateur.

Les requêtes POST sont différentes. Elles ont un URL de destination (comme les requêtes GET) et, en plus, elles contiennent une cargaison (_payload_) qui contient des informations complémentaires. Elles sont en général utilisées pour transmettre les contenus d'un formulaire au serveur, mais pas que.

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.libertyellisfoundation.org/get-search-passenger"

searchString = "match_type=&div_name=&initital_name=&last_name=&result_format=list_view&is_logged_in=&birth_year_from=0&birth_year_to=1974&current_age_from=40&current_age_to=230&arrival_age_from=0&arrival_age_to=140&year_of_arrival_from=0&year_of_arrival_to=1984&month_of_arrival_from=1&month_of_arrival_to=12&day_of_arrival_from=1&day_of_arrival_to=31&town=&ship_name=&port_of_departure=&arrival_port=+&passenger_id=&first_name=&companion_name=&place_of_birth=Paris&ethnicity_search=&search_name=&sort_field=field3&sort_order=asc&submit=Results"

r = requests.post(url, data = {'searchString': searchString})

print(r.text)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

# Après avoir examiné le HTML, on voit que les passagers sont 
# chacun dans un élément DIV de classe "row"
passagers = soup.find_all("div", {"class": "tooltip"})

for passager in passagers:
    divs = passager.find_all("div", {"class": "col6"})
    nom = divs[0].text.replace("NAME OF PASSENGER", "").strip()
    place_of_birth = divs[1].text.replace("PLACE OF BIRTH", "").strip()
    year_arrival = divs[2].text.replace("YEAR OF ARRIVAL", "").strip()
    age_arrival = divs[3].text.replace("AGE AT ARRIVAL", "").strip()
    print(nom, place_of_birth, year_arrival, age_arrival)

In [ ]:
# Exercice: Modifier le code ci-dessus pour scraper les passagers nés dans une autre ville.
# Points bonus si vous arrivez à scraper plusieurs pages de résultats!

## Exemple 2: Airbnb et les APIs internes

Les sites sont souvents construits en plusieurs parties: un site contenant le cadre dans lequel les informations s'affichent (une template) et une API qui permet de charger les informations nécessaires à l'utilisateur. En navigant sur le site en gardant l'onglet 'Network' des outils de développement ouvert, on peut souvent identifier le fonctionnement de l'API.

Comme chez Airbnb.

Pour visualiser un fichier JSON si vous n'avez pas de plugin navigateur: https://www.jsonbeautifier.com/

In [ ]:
url = "https://www.airbnb.com/api/v2/explore_tabs?version=1.3.5&_format=for_explore_search_web&experiences_per_grid=20&items_per_grid=18&guidebooks_per_grid=20&auto_ib=true&fetch_filters=true&has_zero_guest_treatment=false&is_guided_search=true&is_new_cards_experiment=true&luxury_pre_launch=false&query_understanding_enabled=true&show_groupings=true&supports_for_you_v3=true&timezone_offset=120&client_session_id=b7977aa8-ae07-495f-be04-f57ab1d2c6d0&metadata_only=false&is_standard_search=true&refinement_paths%5B%5D=%2Fhomes&selected_tab_id=home_tab&map_toggle=true&allow_override%5B%5D=&ne_lat=47.63778884600719&ne_lng=1.7700038669993319&sw_lat=47.6099605933087&sw_lng=1.4534602878977694&zoom=11&search_by_map=true&federated_search_session_id=91beab26-0411-4e7e-a884-9ebafd40c408&screen_size=medium&query=Blois%2C+France&_intents=p1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=EUR&locale=en"

r = requests.get(url)

print(r.text)

In [ ]:
# Exercice: Créer un scraper Airbnb pour récupérer les informations suivantes dans la ville de votre choix:
# - Nom de l'annonce et son identifiant unique
# - Noms des propriétaires (ou utilisateurs ayant mis en location l'annonce) et leur identifiant unique
# - Prix de la nuité
# - Coordonnées Lat/Lng

# Bonus: Scraper plusieurs pages de résultats

## Exemple 3: Améli Santé et les sessions

Quand un navigateur arrive sur un site, le serveur distant crée une session utilisateur à l'aide de cookies de sessions. Cela permet au serveur de savoir quelles actions un utilisateur a accompli lors de sa visite, par exemple, sans avoir besoin de gérer un système d'inscription / log-in.

Requests permet de gérer les sessions aisément, sans que nous ayons à nous préoccuper des cookies de session et autres.

Démonstration sur http://annuairesante.ameli.fr/

In [ ]:
# Création d'une nouvelle session
s = requests.session()

url = "http://annuairesante.ameli.fr/"

r = s.get(url)

soup = BeautifulSoup(r.text, 'html.parser')

# On extrait le lien qui permet d'accéder à la page de résultat
form = soup.find("div", {"id": "centresite"}).find("form")

lien = form["action"]

print(lien)

In [ ]:
# Le lien complet de la page de résultats
url = "http://annuairesante.ameli.fr" + lien

# Les données de la requête POST
data = {
"type":"ps",
"ps_nom":"+Aa",
"ps_profession":"",
"ps_profession_label":"",
"ps_acte":"",
"ps_acte_label":"",
"ps_type_honoraire":"I_0",
"ps_carte_vitale":"2",
"ps_sexe":"2",
"es_nom":"",
"es_specialite":"",
"es_specialite_label":"",
"es_actes_maladies":"",
"es_actes_maladies_label":"",
"es_type":"3",
"ps_localisation":"",
"ps_proximite":"on",
"localisation_category":"",
"submit_final":"Rechercher"
    }

# On utilise la session ouverte précédement pour faire la requête contenant les résultats.
r = s.post(url, data=data)

print (r.text)

In [ ]:
## Exercice: Extraire les noms des médecins du scraper précédent grâce à BeautifulSoup.